In [ ]:
%matplotlib nbagg

# FA(Firefly Algorithm)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from copy import deepcopy
from plot import graph_plot
from plot import dots_anim_plot

## 目的関数

In [ ]:
def sphere_func(x: list):
    return sum([val ** 2 for val in x])

def rastrigin_func(x: list):
    return sum([val ** 2 - 10 * np.cos(2 * np.pi * val) + 10 for val in x])

## FA method

In [ ]:
def fa(func, M: int=30, D: int=5, tmax: int=1000,
       fend: float=1e-5, xmin: int=-5, xmax: int=5,
       sortver: bool=True):
    """
    @param M <int> : 粒子数
    @param D <int> : 解の次元数
    @param tmax <int> : 最大試行回数
    @param fend <float> : 終了条件
    @param xmin <int> : 初期値の最小値
    @param xmax <int> : 初期値の最大値
    @param sortver <bool> : ソートするかどうか
    ホタルアルゴリズムの実装
    """ 

    # 初期化処理
    # FAのパラメータ
    betamin = 0.2
    alpha0 = 0.5
    alpha = 0
    L = np.fabs(xmax - xmin) / 2
    gamma = 1 / np.sqrt(L)
    # 位置
    x = (xmin - xmax) * np.random.rand(M, D) + xmax
    xnew = np.zeros((M, D))
    # 解の評価値関数
    f = np.zeros(M)
    I = np.zeros(M)
    # 目的関数の値
    xbest = np.zeros(D)
    fbest = float("inf")
    rij = float("inf")
    # その他のパラメータ
    k = 0
    # 可視化用
    xs = []
    fs = []
    
    # 関数の初期値の終了
    for i in range(M):
        f[i] = func(x[i])
        fbest = f[i] if f[i] < fbest else fbest
    
    # 実行
    for t in range(tmax):
        # aの更新
        alpha = alpha0 * ((1e-4 / 0.9) ** ((t + 1) / tmax))
        e = (np.random.rand() - 0.5) * L
        xnew = deepcopy(x)
        for i in range(M):
            f[i] = func(x[i])
            I[i] = 1 / f[i]
        
        # ソートをかけて光強度が低い順にすすめていく
        if sortver:
            # sort
            k = 0
            tmp = list(zip(I, xnew))
            I, xnew = zip(*sorted(tmp, key=lambda a:a[0], reverse=True))
            I = np.array(I)
            xnew = np.array(xnew)
            for i in range(1, M)[::-1]:
                for j in range(0, i)[::-1]:
                    rij = np.linalg.norm(xnew[i] - x[j])
                    beta = (1 - betamin) * np.exp(-gamma * (rij ** 2)) + betamin
                    for l in range(D):
                        e = (np.random.rand() - 0.5) * L
                        xnew[i][l] += beta * (x[j][l] - xnew[i][l]) + alpha * e
            for l in range(D):
                e = (np.random.rand() - 0.5) * L
                xnew[0][l] += alpha * e
        # 順に光強度を参照していき，強い光に惹きつけられる
        else:
            # normal
            for i in range(M):
                cnt = 0
                for j in range(M):
                    if I[i] < I[j]:
                        cnt += 1
                        rij = np.linalg.norm(xnew[i] - x[j])
                        beta = (1 - betamin) * np.exp(-gamma * (rij ** 2)) + betamin
                        for l in range(D):
                            e = (np.random.rand() - 0.5) * L
                            xnew[i][l] += beta * (x[j][l] - xnew[i][l]) + alpha * e
                if cnt == 0:
                    k = i
                    for l in range(D):
                        e = (np.random.rand() - 0.5) * L
                        xnew[k][l] += alpha * e

        # 現在の最適解よりもいい解なら
        if f[k] < fbest:
            fbest = f[k]
            xbest = x[k]
        # 可視化用にパラメータを保持
        xs.append(xnew)
        fs.append(fbest)
        # 終了条件を満たしていたら
        if fbest < fend:
            break
        # ホタルの位置情報を更新
        x = deepcopy(xnew)
        
    return t + 1, fbest, xbest, fs, xs

### 動作テスト＋可視化

In [ ]:
a, b, c, d, e = fa(rastrigin_func, D=5, tmax=100000, M=100)
print(a, b, c)

In [ ]:
t, f, x, fs, xs = fa(sphere_func, D=2, sortver=False)
dots_anim_plot(xs)
graph_plot(t + 1, fs)

### 実験(100回実行)

In [ ]:
sphere2 = [fa(func=sphere_func, D=2, sortver=False) for _ in range(100)]
rastrigin2 = [fa(func=rastrigin_func, D=2, sortver=False) for _ in range(100)]
sphere5 = [fa(func=sphere_func, D=5, sortver=False) for _ in range(100)]
rastrigin5 = [fa(func=rastrigin_func, D=5, sortver=False) for _ in range(100)]
sphere20 = [fa(func=sphere_func, D=20, sortver=False) for _ in range(100)]
rastrigin20 = [fa(func=rastrigin_func, D=20, sortver=False) for _ in range(100)]

In [ ]:
sphere2_time = [s[0] for s in sphere2]
sphere2_result = [s[1] for s in sphere2]
sphere5_time = [s[0] for s in sphere5]
sphere5_result = [s[1] for s in sphere5]
sphere20_time = [s[0] for s in sphere20]
sphere20_result = [s[1] for s in sphere20]

rastrigin2_time = [r[0] for r in rastrigin2]
rastrigin2_result = [r[1] for r in rastrigin2]
rastrigin5_time = [r[0] for r in rastrigin5]
rastrigin5_result = [r[1] for r in rastrigin5]
rastrigin20_time = [r[0] for r in rastrigin20]
rastrigin20_result = [r[1] for r in rastrigin20]

In [ ]:
sphere2_time_mean = np.mean(sphere2_time)
sphere2_result_mean = np.mean(sphere2_result)
sphere2_result_var = np.var(sphere2_result)
sphere5_time_mean = np.mean(sphere5_time)
sphere5_result_mean = np.mean(sphere5_result)
sphere5_result_var = np.var(sphere5_result)
sphere20_time_mean = np.mean(sphere20_time)
sphere20_result_mean = np.mean(sphere20_result)
sphere20_result_var = np.var(sphere20_result)

rastrigin2_time_mean = np.mean(rastrigin2_time)
rastrigin2_result_mean = np.mean(rastrigin2_result)
rastrigin2_result_var = np.var(rastrigin2_result)
rastrigin5_time_mean = np.mean(rastrigin5_time)
rastrigin5_result_mean = np.mean(rastrigin5_result)
rastrigin5_result_var = np.var(rastrigin5_result)
rastrigin20_time_mean = np.mean(rastrigin20_time)
rastrigin20_result_mean = np.mean(rastrigin20_result)
rastrigin20_result_var = np.var(rastrigin20_result)

In [ ]:
print(sphere2_result_mean, sphere2_result_var, sphere2_time_mean)
print(rastrigin2_result_mean, rastrigin2_result_var, rastrigin2_time_mean)
print(sphere5_result_mean, sphere5_result_var, sphere5_time_mean)
print(rastrigin5_result_mean, rastrigin5_result_var, rastrigin5_time_mean)
print(sphere20_result_mean, sphere20_result_var, sphere20_time_mean)
print(rastrigin20_result_mean, rastrigin20_result_var, rastrigin20_time_mean)